# Construir um modelo de classificação: Deliciosas culinárias asiáticas e indianas


## Classificadores de culinária 2

Nesta segunda lição de classificação, exploraremos `mais maneiras` de classificar dados categóricos. Também aprenderemos sobre as implicações de escolher um classificador em vez de outro.

### [**Quiz pré-aula**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **Pré-requisitos**

Assumimos que você completou as lições anteriores, já que continuaremos com alguns conceitos que aprendemos antes.

Para esta lição, precisaremos dos seguintes pacotes:

-   `tidyverse`: O [tidyverse](https://www.tidyverse.org/) é uma [coleção de pacotes R](https://www.tidyverse.org/packages) projetada para tornar a ciência de dados mais rápida, fácil e divertida!

-   `tidymodels`: O [tidymodels](https://www.tidymodels.org/) é um [framework de pacotes](https://www.tidymodels.org/packages/) para modelagem e aprendizado de máquina.

-   `themis`: O [pacote themis](https://themis.tidymodels.org/) fornece etapas extras de receitas para lidar com dados desbalanceados.

Você pode instalá-los com o seguinte comando:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

Alternativamente, o script abaixo verifica se você possui os pacotes necessários para completar este módulo e os instala caso estejam ausentes.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. Um mapa de classificação**

Na nossa [lição anterior](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1), tentamos responder à pergunta: como escolher entre vários modelos? Em grande parte, isso depende das características dos dados e do tipo de problema que queremos resolver (por exemplo, classificação ou regressão?).

Anteriormente, aprendemos sobre as várias opções disponíveis para classificar dados usando o guia da Microsoft. O framework de Machine Learning do Python, Scikit-learn, oferece um guia semelhante, mas mais detalhado, que pode ajudar ainda mais a restringir seus estimadores (outro termo para classificadores):

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> Dica: [visite este mapa online](https://scikit-learn.org/stable/tutorial/machine_learning_map/) e clique ao longo do caminho para ler a documentação.
>
> O [site de referência do Tidymodels](https://www.tidymodels.org/find/parsnip/#models) também oferece uma excelente documentação sobre diferentes tipos de modelos.

### **O plano** 🗺️

Este mapa é muito útil quando você tem uma compreensão clara dos seus dados, pois pode 'caminhar' pelos seus caminhos até chegar a uma decisão:

-   Temos \>50 amostras

-   Queremos prever uma categoria

-   Temos dados rotulados

-   Temos menos de 100K amostras

-   ✨ Podemos escolher um Linear SVC

-   Se isso não funcionar, já que temos dados numéricos

    -   Podemos tentar um ✨ KNeighbors Classifier

        -   Se isso não funcionar, tente ✨ SVC e ✨ Ensemble Classifiers

Este é um caminho muito útil para seguir. Agora, vamos direto ao ponto usando o framework de modelagem [tidymodels](https://www.tidymodels.org/): uma coleção consistente e flexível de pacotes R desenvolvidos para incentivar boas práticas estatísticas 😊.

## 2. Divida os dados e lide com o conjunto de dados desequilibrado.

Nas nossas lições anteriores, aprendemos que havia um conjunto de ingredientes comuns entre nossas culinárias. Além disso, havia uma distribuição bastante desigual no número de culinárias.

Vamos lidar com isso da seguinte forma:

-   Eliminando os ingredientes mais comuns que criam confusão entre culinárias distintas, usando `dplyr::select()`.

-   Usando uma `recipe` que pré-processa os dados para prepará-los para modelagem, aplicando um algoritmo de `over-sampling`.

Já vimos isso na lição anterior, então deve ser tranquilo 🥳!


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### Lidando com dados desbalanceados

Dados desbalanceados frequentemente têm efeitos negativos no desempenho do modelo. Muitos modelos funcionam melhor quando o número de observações é igual e, por isso, tendem a ter dificuldades com dados desbalanceados.

Existem basicamente duas maneiras de lidar com conjuntos de dados desbalanceados:

-   adicionar observações à classe minoritária: `Over-sampling`, por exemplo, usando um algoritmo SMOTE que gera novos exemplos sintéticos da classe minoritária utilizando os vizinhos mais próximos desses casos.

-   remover observações da classe majoritária: `Under-sampling`

Na nossa aula anterior, demonstramos como lidar com conjuntos de dados desbalanceados usando um `recipe`. Um recipe pode ser pensado como um plano que descreve quais etapas devem ser aplicadas a um conjunto de dados para prepará-lo para análise. No nosso caso, queremos ter uma distribuição igual no número de nossas culinárias para o nosso `training set`. Vamos direto ao ponto.


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

Agora estamos prontos para treinar modelos 👩‍💻👨‍💻!

## 3. Além dos modelos de regressão multinomial

Na nossa aula anterior, exploramos os modelos de regressão multinomial. Agora, vamos conhecer alguns modelos mais flexíveis para classificação.

### Máquinas de Vetores de Suporte

No contexto de classificação, `Máquinas de Vetores de Suporte` é uma técnica de aprendizado de máquina que busca encontrar um *hiperplano* que "melhor" separa as classes. Vamos observar um exemplo simples:

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ não separa as classes. H2~ separa, mas apenas com uma margem pequena. H3~ as separa com a margem máxima.

#### Classificador Linear de Vetores de Suporte

O agrupamento por Vetores de Suporte (SVC) é um membro da família de técnicas de aprendizado de máquina baseadas em Máquinas de Vetores de Suporte. No SVC, o hiperplano é escolhido para separar corretamente `a maioria` das observações de treinamento, mas `pode classificar erroneamente` algumas observações. Ao permitir que alguns pontos fiquem do lado errado, o SVM se torna mais robusto a outliers, o que resulta em uma melhor generalização para novos dados. O parâmetro que regula essa violação é chamado de `cost`, que tem um valor padrão de 1 (veja `help("svm_poly")`).

Vamos criar um SVC linear definindo `degree = 1` em um modelo SVM polinomial.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

Agora que capturamos as etapas de pré-processamento e a especificação do modelo em um *workflow*, podemos prosseguir para treinar o SVC linear e avaliar os resultados ao mesmo tempo. Para as métricas de desempenho, vamos criar um conjunto de métricas que avaliará: `accuracy`, `sensitivity`, `Positive Predicted Value` e `F Measure`.

> `augment()` adicionará coluna(s) de previsões aos dados fornecidos.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### Máquina de Vetores de Suporte

A máquina de vetores de suporte (SVM) é uma extensão do classificador de vetores de suporte para acomodar um limite não linear entre as classes. Essencialmente, as SVMs utilizam o *truque do kernel* para ampliar o espaço de características e se adaptar a relações não lineares entre as classes. Uma função kernel popular e extremamente flexível usada pelas SVMs é a *função de base radial.* Vamos ver como ela se comporta com nossos dados.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Muito melhor 🤩!

> ✅ Por favor, veja:
>
> -   [*Support Vector Machines*](https://bradleyboehmke.github.io/HOML/svm.html), Hands-on Machine Learning with R
>
> -   [*Support Vector Machines*](https://www.statlearning.com/), An Introduction to Statistical Learning with Applications in R
>
> para leitura adicional.

### Classificadores de Vizinhos Mais Próximos

O algoritmo *K*-nearest neighbor (KNN) prevê cada observação com base em sua *semelhança* com outras observações.

Vamos ajustar um ao nosso conjunto de dados.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Parece que este modelo não está apresentando um bom desempenho. Provavelmente, alterar os argumentos do modelo (veja `help("nearest_neighbor")`) pode melhorar o desempenho do modelo. Certifique-se de testar isso.

> ✅ Por favor, veja:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> para aprender mais sobre classificadores *K*-Nearest Neighbors.

### Classificadores de conjunto

Os algoritmos de conjunto funcionam combinando múltiplos estimadores base para produzir um modelo ideal, seja por:

`bagging`: aplicando uma *função de média* a uma coleção de modelos base

`boosting`: construindo uma sequência de modelos que se baseiam uns nos outros para melhorar o desempenho preditivo.

Vamos começar experimentando um modelo de Random Forest, que constrói uma grande coleção de árvores de decisão e, em seguida, aplica uma função de média para obter um modelo geral melhor.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Bom trabalho 👏!

Vamos também experimentar com um modelo de Árvore Reforçada.

A Árvore Reforçada define um método de conjunto que cria uma série de árvores de decisão sequenciais, onde cada árvore depende dos resultados das árvores anteriores, na tentativa de reduzir o erro de forma incremental. Ela foca nos pesos dos itens classificados incorretamente e ajusta o ajuste para o próximo classificador corrigir.

Existem diferentes maneiras de ajustar este modelo (veja `help("boost_tree")`). Neste exemplo, ajustaremos Árvores Reforçadas usando o mecanismo `xgboost`.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ Por favor, veja:
>
> -   [Machine Learning for Social Scientists](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - Explora o modelo AdaBoost, que é uma boa alternativa ao xgboost.
>
> para aprender mais sobre classificadores Ensemble.

## 4. Extra - comparando múltiplos modelos

Nós ajustamos um bom número de modelos neste laboratório 🙌. Pode se tornar cansativo ou trabalhoso criar muitos fluxos de trabalho a partir de diferentes conjuntos de pré-processadores e/ou especificações de modelos e, em seguida, calcular as métricas de desempenho uma por uma.

Vamos ver se conseguimos resolver isso criando uma função que ajusta uma lista de fluxos de trabalho no conjunto de treinamento e, em seguida, retorna as métricas de desempenho com base no conjunto de teste. Vamos usar `map()` e `map_dfr()` do pacote [purrr](https://purrr.tidyverse.org/) para aplicar funções a cada elemento de uma lista.

> As funções [`map()`](https://purrr.tidyverse.org/reference/map.html) permitem substituir muitos loops `for` por um código que é mais conciso e mais fácil de ler. O melhor lugar para aprender sobre as funções [`map()`](https://purrr.tidyverse.org/reference/map.html) é o [capítulo de iteração](http://r4ds.had.co.nz/iteration.html) em R for Data Science.


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


O pacote [**workflowset**](https://workflowsets.tidymodels.org/) permite que os usuários criem e ajustem facilmente um grande número de modelos, mas é principalmente projetado para funcionar com técnicas de reamostragem, como `cross-validation` (validação cruzada), uma abordagem que ainda vamos abordar.

## **🚀Desafio**

Cada uma dessas técnicas possui um grande número de parâmetros que você pode ajustar, como, por exemplo, `cost` em SVMs, `neighbors` em KNN, `mtry` (Preditores Selecionados Aleatoriamente) em Random Forest.

Pesquise os parâmetros padrão de cada uma e reflita sobre o que ajustar esses parâmetros significaria para a qualidade do modelo.

Para saber mais sobre um modelo específico e seus parâmetros, use: `help("model")`, por exemplo, `help("rand_forest")`.

> Na prática, geralmente *estimamos* os *melhores valores* para esses parâmetros treinando muitos modelos em um `conjunto de dados simulado` e medindo o desempenho de todos esses modelos. Esse processo é chamado de **tuning** (ajuste fino).

### [**Quiz pós-aula**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **Revisão e Autoestudo**

Há muitos termos técnicos nessas lições, então reserve um momento para revisar [esta lista](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) de terminologia útil!

#### AGRADECIMENTOS A:

[`Allison Horst`](https://twitter.com/allison_horst/) por criar as ilustrações incríveis que tornam o R mais acolhedor e envolvente. Encontre mais ilustrações na sua [galeria](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Cassie Breviu](https://www.twitter.com/cassieview) e [Jen Looper](https://www.twitter.com/jenlooper) por criarem a versão original deste módulo em Python ♥️

Bons estudos,

[Eric](https://twitter.com/ericntay), Embaixador Estudante Gold da Microsoft Learn.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>Arte por @allison_horst</figcaption>



---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, esteja ciente de que traduções automatizadas podem conter erros ou imprecisões. O documento original em seu idioma nativo deve ser considerado a fonte autoritativa. Para informações críticas, recomenda-se a tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações equivocadas decorrentes do uso desta tradução.
